### **Importar de Librerías**

In [2]:
import pandas as pd
import ast
import json

### **Cargar movies_dataset.csv**

In [3]:
# cargar movie_dataset en un dataframe de pandas
df_movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies_dataset.csv')

### **Visualizar del dataset**

In [ ]:
df_movies.info()

In [ ]:
df_movies.head()

In [ ]:
df_movies.describe()

### **Detectar y arreglar problemas con columnas divididas**

In [ ]:
# identificar id que no sean numéricos]
# convertir la columna 'id' a numérica y asignar NaN a los valores no enteros
df_movies['id_numeric'] = pd.to_numeric(df_movies['id'], errors='coerce')
# filtrar los valores no enteros
non_integer_values = df_movies[df_movies['id_numeric'].isnull()]
# imprimir los valores no enteros
print(non_integer_values)
df_movies = df_movies.drop('id_numeric', axis=1) #Eliminar la columna 'id_numeric' para dejar las columnas originales

In [4]:
# función para corregir el error de los registros que están divididos en dos filas
def merge_and_delete(df, indices):
    new_df = df.copy()  # copia el DataFrame original para evitar modificarlo directamente

    for index in indices:
        # obtener valores a modificar
        value1 = new_df.iloc[index, 0]
        value2 = new_df.iloc[index-1, 9]

        # concatenar los valores
        concatenated_value = str(value2) + str(value1)

        # actualizar el valor de la columna 10 del registro anterior con el resultado concatenado
        new_df.iloc[index-1, 9] = concatenated_value

        # copiar los valores del registro actual en las columnas 10 a 23 del registro anterior
        values = new_df.iloc[index, 1:14].values
        new_df.iloc[index-1, 10:23] = values

    # eliminar los registros indicados
    new_df = new_df.drop(indices)

    # reindexar el dataframe
    new_df = new_df.reset_index(drop=True)
    return new_df

In [5]:
# corregir los registros divididos en dos filas identificados anteriormente
indices = [19730, 29503, 35587]
df_movies = merge_and_delete(df_movies, indices)

### **Desanidar columnas que contienen diccionarios**

In [6]:
# funcion para desanidar diccionarios
def desanidar_columnas(df, columnas):
    for columna in columnas:
        # convertir la columna en una lista de diccionarios
        df[columna] = df[columna].apply(lambda x: ast.literal_eval(x))

        # cbtener los valores de cada diccionario y manejar las celdas vacías
        df[columna] = df[columna].apply(lambda x: [item['name'] for item in x] if x else ['sin dato'])

        # rellenar las celdas vacías con ['sin dato']
        df[columna] = df[columna].apply(lambda x: x if x else ['sin dato'])

In [7]:
# desanidar la columna 'belong_to_collection'
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

# crear columnas separadas
df_movies['id_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('id', 0))
df_movies['name_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('name', 'sin dato'))
df_movies['poster_path_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('poster_path', 'sin dato'))
df_movies['backdrop_path_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('backdrop_path', 'sin dato'))

# eliminar la columna original 'belong_to_collection'
df_movies.drop('belongs_to_collection', axis=1, inplace=True)

In [8]:
# desanidar las otras columnas con diccionarios
columnas_a_desanidar = ['genres', 'production_companies', 'production_countries', 'spoken_languages']
desanidar_columnas(df_movies, columnas_a_desanidar)

In [ ]:
df_movies.columns

### **Identificar y eliminar duplicados**

In [9]:
# lista de columnas a convertir
columns_to_convert = ['genres', 'production_companies', 'production_countries', 'spoken_languages']

# convertir listas a tuplas para poder identificar duplicados
df_movies[columns_to_convert] = df_movies[columns_to_convert].applymap(lambda x: tuple(x) if isinstance(x, (list)) else x)

# contar duplicados
df_movies.duplicated().sum()

13

In [10]:
# eliminar duplicados
df_movies = df_movies.drop_duplicates()

# reindexar el dataframe
df_movies = df_movies.reset_index(drop=True)

### **Remover columnas indicadas en el enunciado del proyecto**

In [11]:
# remover las columnas del DataFrame solicitadas en el enunciado del PI
columnas_a_remover = ['video','imdb_id','adult','original_title','poster_path','homepage']
df_movies = df_movies.drop(columnas_a_remover, axis=1)

### **Adecuar tipo de columnas de acuerdo a los datos que contienen**

In [12]:
# convertir las columnas 'budget', 'revenue' y 'runtime' a tipo numérico (float)
columnas_a_numeric = ['budget', 'revenue', 'runtime', 'popularity', 'vote_average']
df_movies[columnas_a_numeric] = df_movies[columnas_a_numeric].apply(pd.to_numeric)

In [13]:
# convertir columna 'release_date' a formato fecha
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')

### **Tratar valores nulos**

In [ ]:
# obtener las columnas con valores nulos
columnas_nulas = df_movies.columns[df_movies.isnull().any()]

# obtener los tipos de datos de las columnas con valores nulos
tipos_columnas_nulas = df_movies[columnas_nulas].dtypes

# mostrar los tipos de datos de las columnas con valores nulos
print(tipos_columnas_nulas)

In [14]:
# lista de columnas no numéricas en las que se reemplazarán los valores nulos
columnas_sin_dato = ['original_language','overview','status','tagline','poster_path_collection','backdrop_path_collection']

# completar los valores nulos en las columnas seleccionadas con 'sin dato'
df_movies[columnas_sin_dato] = df_movies[columnas_sin_dato].fillna('sin dato')

# lista de columnas numéricas en las que se reemplazarán los valores nulos
columnas_cero = ['runtime','vote_count']

# completar los valores nulos en las columnas seleccionadas con 'sin dato'
df_movies[columnas_cero] = df_movies[columnas_cero].fillna(0)

In [15]:
# eliminar registros con el campo 'release_date' nulo
df_movies.dropna(subset=['release_date'], inplace=True)

### **Crear columnas solicitadas en el enunciado del proyecto**

In [16]:
# crear la columna "return" para guardar el retorno de inversión
df_movies['return'] = 0  # Inicializar la columna "return" con valor 0

# calcular el retorno de inversión solo cuando "budget" sea diferente de 0
mask = df_movies['budget'] != 0
df_movies.loc[mask, 'return'] = df_movies.loc[mask, 'revenue'] / df_movies.loc[mask, 'budget']

In [17]:
# extraer el año y guardarlo en la columna 'release_year'
df_movies['release_year'] = pd.to_datetime(df_movies['release_date']).apply(lambda x: x.year)

### **Convertir a *integer* algunas columnas**

In [18]:
# lista de columnas que se convertirán a int
columnas_a_int = ['id', 'vote_count', 'id_collection']

# convertir las columnas seleccionadas de float a int
df_movies[columnas_a_int] = df_movies[columnas_a_int].astype('int64')

### **Cargar credits.csv**

In [19]:
# cargar credits en un dataframe de pandas
df_credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credits.csv')

### **Visualizar dataset**

In [ ]:
df_credits.info()

In [ ]:
df_credits.head()

### **Desanidar y convertir diccionarios de columnas**

In [20]:
# desanidar la columna 'cast'
desanidar_columnas(df_credits, ['cast'])

In [21]:
# convertir la columna 'crew' de cadena a lista de diccionarios
df_credits['crew'] = df_credits['crew'].apply(ast.literal_eval)

# crear la columna 'director' y eliminar la columna 'crew'
df_credits['director'] = df_credits['crew'].apply(lambda x: [item['name'] for item in x if item['job'] == 'Director'][0] if any(item['job'] == 'Director' for item in x) else 'sin dato')
df_credits = df_credits.drop('crew', axis=1)

### **Identificar y eliminar duplicados**

In [23]:
# lista de nombres de columnas a convertir
columns_to_convert = ['cast', 'director']

# convertir listas a tuplas para identificar duplicados
df_credits[columns_to_convert] = df_credits[columns_to_convert].applymap(lambda x: tuple(x) if isinstance(x, (list)) else x)

# contar duplicados
df_credits.duplicated().sum()

43

In [24]:
# elimimar duplicados
df_credits = df_credits.drop_duplicates()

# reindexar el dataframe
df_credits = df_credits.reset_index(drop=True)

### **Tratar valores nulos**

In [25]:
# obtener las columnas con valores nulos
columnas_nulas = df_credits.columns[df_credits.isnull().any()]
columnas_nulas

Index([], dtype='object')

No se encontraron valores nulos en este dataset

In [ ]:
# obtener la longitud de la lista más larga en la columna de actores
max(len(actor_list) for actor_list in df_credits['cast'])

In [29]:
# IMDB considera reparto principal los tres primeros actores, de modo que las listas se reducirán a un máximo de 5 actores.
df_credits['cast'] = df_credits['cast'].apply(lambda x: x[:5])

### **Unir los dataframes movies y credits**

In [31]:
df = pd.merge(df_movies, df_credits, on='id')

### **Guardar el csv completo y limpio**

In [32]:
# especificar la ruta y el nombre de archivo para guardar el archivo CSV
csv_file = "/content/drive/MyDrive/Colab Notebooks/movies_credits_limpio.csv"

# guardar el DataFrame en un archivo CSV sin comillas y con el formato de fecha específico
df.to_csv(csv_file, date_format='%Y-%m-%d', index=False)

print("Archivo CSV guardado exitosamente.")

Archivo CSV guardado exitosamente.
